<a href="https://www.kaggle.com/code/fhloston/the-fremen?scriptVersionId=182262897" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import pandas as pd

import pandasql as psql

file_path = '/kaggle/input/ipo-stock/IPO.csv'

IPO = pd.read_csv(file_path)

IPO.head()

,IPODate,Symbol,CompanyName,IPOPrice,Current,Return,MarketCap,DealSize,Exchange
0,7-Jun-24,WAY,Waystar Holding Corp.,$21.50,$20.70,-3.72%,3.45B,967.50M,NASDAQ
1,7-Jun-24,RAPP,"Rapport Therapeutics, Inc.",$17.00,$20.80,22.35%,735.83M,136.00M,NASDAQ
2,7-Jun-24,CHEB,Chenghe Acquisition II Co.,$10.00,$10.02,0.20%,100.20M,75.00M,NYSEAMERICAN
3,6-Jun-24,GAUZ,Gauzy Ltd.,$17.00,$15.74,-7.41%,295.29M,75.00M,NASDAQ
4,6-Jun-24,FLYE,"Fly-E Group, Inc.",$4.00,$4.00,NaN,100.00M,9.00M,NASDAQ


# SPAC Companies

In [2]:
start_date = '6-Jun-00'
end_date = '8-Jun-24'

query = f"""
Select *
from IPO
where CompanyName like '%Acquisition%'
and IPODate between '{start_date}' AND '{end_date}'
"""

result = psql.sqldf(query, locals())
display(result)

,IPODate,Symbol,CompanyName,IPOPrice,Current,Return,MarketCap,DealSize,Exchange
0,7-Jun-24,CHEB,Chenghe Acquisition II Co.,$10.00,$10.02,0.20%,100.20M,75.00M,NYSEAMERICAN
1,6-Oct-23,SPKL,Spark I Acquisition Corp. Class A Ordinary Share,$10.00,$10.34,3.40%,169.80M,100.00M,NASDAQ
2,6-Oct-23,QETA,Quetta Acquisition Corporation,$10.00,$10.34,3.35%,92.47M,60.00M,NASDAQ
3,7-Apr-22,DECA,Denali Capital Acquisition Corp.,$10.00,$7.00,-30.00%,49.77M,75.00M,NASDAQ
4,8-Feb-22,LIVB,LIV Capital Acquisition Corp. II,$10.00,$10.97,9.70%,127.36M,100.00M,NASDAQ
5,7-Jan-22,CXAC,C5 Acquisition Corporation,$10.00,$10.74,7.40%,387.59M,250.00M,NYSE
6,7-Jan-22,VBOC,Viscogliosi Brothers Acquisition Corp.,$10.00,$10.48,4.75%,112.88M,75.00M,NASDAQ
7,7-Apr-22,DECA,Denali Capital Acquisition Corp.,$10.00,$7.00,-30.00%,49.77M,75.00M,NASDAQ
8,8-Feb-22,LIVB,LIV Capital Acquisition Corp. II,$10.00,$10.97,9.70%,127.36M,100.00M,NASDAQ
9,7-Jan-22,CXAC,C5 Acquisition Corporation,$10.00,$10.74,7.40%,387.59M,250.00M,NYSE


# REIT Companies

In [3]:
start_date = '6-Jun-00'
end_date = '8-Jun-24'

query = f"""
Select * 
from IPO 
where CompanyName like '%Trust%' 
and IPODate between '{start_date}' AND '{end_date}'
"""

result = psql.sqldf(query, locals())
display(result)

,IPODate,Symbol,CompanyName,IPOPrice,Current,Return,MarketCap,DealSize,Exchange
0,7-Oct-20,SQFT,"Presidio Property Trust, Inc.",$5.00,$0.82,-83.70%,11.86M,2.50M,NASDAQ


# MIN/MAX IPOPrice

In [4]:
start_date = '6-Jun-00'
end_date = '8-Jun-24'

query = f"""
Select Symbol, CompanyName, IPODate, max(IPOPrice), min(IPOPrice) 
from IPO
where IPODate between '{start_date}' AND '{end_date}'
group by Symbol, CompanyName, IPODate, Exchange
"""

result = psql.sqldf(query, locals())
display(result)

,Symbol,CompanyName,IPODate,max(IPOPrice),min(IPOPrice)
0,WAY,Waystar Holding Corp.,7-Jun-24,$21.50,$21.50
1,AHR,"American Healthcare REIT, Inc.",7-Feb-24,$12.00,$12.00
2,ALEC,"Alector, Inc.",7-Feb-19,$19.00,$19.00
3,AMLX,"Amylyx Pharmaceuticals, Inc.",7-Jan-22,$19.00,$19.00
4,ARYA,ARYA Sciences Acquisition Corp. III,7-Aug-20,$10.00,$10.00
...,...,...,...,...,...
71,TRVI,"Trevi Therapeutics, Inc.",7-May-19,$10.00,$10.00
72,TXAC,Therapeutics Acquisition Corp.,8-Jul-20,$10.00,$10.00
73,VBOC,Viscogliosi Brothers Acquisition Corp.,7-Jan-22,$10.00,$10.00
74,VIGL,"Vigil Neuroscience, Inc.",7-Jan-22,$14.00,$14.00


# MIN/MAX MarketCap

In [5]:
start_date = '6-Jun-00'
end_date = '8-Jun-24'

query = f"""
Select Symbol, CompanyName, IPODate, min(MarketCap), max(MarketCap) 
from IPO 
where IPODate between '{start_date}' AND '{end_date}'
group by Symbol, CompanyName, IPODate, Exchange
"""

result = psql.sqldf(query, locals())
display(result)

,Symbol,CompanyName,IPODate,min(MarketCap),max(MarketCap)
0,WAY,Waystar Holding Corp.,7-Jun-24,3.45B,3.45B
1,AHR,"American Healthcare REIT, Inc.",7-Feb-24,763.18M,763.18M
2,ALEC,"Alector, Inc.",7-Feb-19,420.24M,420.24M
3,AMLX,"Amylyx Pharmaceuticals, Inc.",7-Jan-22,120.37M,120.37M
4,ARYA,ARYA Sciences Acquisition Corp. III,7-Aug-20,None,None
...,...,...,...,...,...
71,TRVI,"Trevi Therapeutics, Inc.",7-May-19,171.86M,171.86M
72,TXAC,Therapeutics Acquisition Corp.,8-Jul-20,None,None
73,VBOC,Viscogliosi Brothers Acquisition Corp.,7-Jan-22,112.88M,112.88M
74,VIGL,"Vigil Neuroscience, Inc.",7-Jan-22,172.89M,172.89M


# Big Deals


In [6]:
start_date = '6-Jun-00'
end_date = '8-Jun-24'

query = f"""
Select Symbol, CompanyName, IPODate, DealSize
from IPO 
where IPODate between '{start_date}' and '{end_date}'
and DealSize >= '100M'
"""

result = psql.sqldf(query, locals())
display(result)

,Symbol,CompanyName,IPODate,DealSize
0,WAY,Waystar Holding Corp.,7-Jun-24,967.50M
1,RAPP,"Rapport Therapeutics, Inc.",7-Jun-24,136.00M
2,CHEB,Chenghe Acquisition II Co.,7-Jun-24,75.00M
3,GAUZ,Gauzy Ltd.,6-Jun-24,75.00M
4,FLYE,"Fly-E Group, Inc.",6-Jun-24,9.00M
5,RYDE,Ryde Group Ltd.,6-Mar-24,12.00M
6,KYTX,"Kyverna Therapeutics, Inc.",8-Feb-24,319.00M
7,PMNT,Perfect Moment Ltd.,8-Feb-24,8.00M
8,AHR,"American Healthcare REIT, Inc.",7-Feb-24,672.00M
9,QETA,Quetta Acquisition Corporation,6-Oct-23,60.00M


# MIN/MAX Big Deals

In [7]:
start_date = '6-Jun-00'
end_date = '8-Jun-24'

query = f"""
Select Symbol, CompanyName, IPODate, min(DealSize), max(DealSize) 
from IPO 
where IPODate between '{start_date}' and '{end_date}'
and DealSize >= '100M'
group by Symbol, CompanyName, IPODate, Exchange
"""

result = psql.sqldf(query, locals())
display(result)

,Symbol,CompanyName,IPODate,min(DealSize),max(DealSize)
0,WAY,Waystar Holding Corp.,7-Jun-24,967.50M,967.50M
1,AHR,"American Healthcare REIT, Inc.",7-Feb-24,672.00M,672.00M
2,AMLX,"Amylyx Pharmaceuticals, Inc.",7-Jan-22,190.00M,190.00M
3,ATAK,DIH Holding US Inc,7-Feb-22,200.00M,200.00M
4,BLCO,Bausch + Lomb Corporation,6-May-22,630.00M,630.00M
5,CHEB,Chenghe Acquisition II Co.,7-Jun-24,75.00M,75.00M
6,CINC,"CinCor Pharma, Inc.",7-Jan-22,193.60M,193.60M
7,CXAC,C5 Acquisition Corporation,7-Jan-22,250.00M,250.00M
8,DECA,Denali Capital Acquisition Corp.,7-Apr-22,75.00M,75.00M
9,FLYE,"Fly-E Group, Inc.",6-Jun-24,9.00M,9.00M


# Exchange Statistics

In [8]:
start_date = '6-Jun-00'
end_date = '8-Jun-24'

query = f"""
Select Exchange, count(Symbol)
from IPO 
where IPODate between '{start_date}' and '{end_date}'
"""

result = psql.sqldf(query, locals())
display(result)

,Exchange,count(Symbol)
0,NASDAQ,89


# Exchange with biggest IPO count

In [9]:
start_date = '6-Jun-00'
end_date = '8-Jun-24'

query = f"""
Select Exchange, count(Symbol)
from IPO 
where IPODate between '{start_date}' and '{end_date}'
group by Exchange
order by count(Symbol) desc
Limit 1
"""

result = psql.sqldf(query, locals())
display(result)

,Exchange,count(Symbol)
0,NASDAQ,45


# IPO Positive return

In [10]:
start_date = '6-Jun-00'
end_date = '8-Jun-24'

query = f"""
SELECT COUNT(Symbol) AS PositiveReturnCount 
FROM IPO 
WHERE Return > 0
"""

result = psql.sqldf(query, locals())
display(result)

,PositiveReturnCount
0,571


Total market capitalization